In [2]:
import pandas as pd 
import numpy as np
import tweet_sentiment as ts

import json

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
No module named 'numpy.char'

In [ ]:
with open("../data/scraped_data.json") as f:
    data = json.load(f)
btc_data = filter(lambda x: x["ticker"] == "BTC", data)
btc_data = list(btc_data)

In [ ]:
def load_prices(path="../data/Bitcoin 2024.csv"):
    price_data = pd.read_csv(path)

    price_data["date"] = pd.to_datetime(price_data["Date"]).astype('datetime64[ns]')
    price_data["price"] = price_data["Price"].str.replace(",", "").astype(float)
    price_data["change"] = price_data["Change %"].str.replace("%", "").astype(float)
    price_data = price_data[['date', 'price', 'change']]

    return price_data

price_data = load_prices()
price_data.head()

,date,price,change
0,2024-11-25,98600.6,0.68
1,2024-11-24,97935.2,0.24
2,2024-11-23,97699.0,-1.24
3,2024-11-22,98929.7,0.56
4,2024-11-21,98374.5,4.32


In [ ]:
# Process each BTC tweet and store results in a list
tweet_data = []
TICKER = "btc"
for tweet_group in btc_data:
    for tweet in tweet_group["tweets"]:
        # Get sentiment analysis results
        main_post, sentiment_results = ts.process_tweet_sentiments(tweet)

        # Only process if BTC was mentioned
        if TICKER in sentiment_results:
            btc_sentiments = sentiment_results[TICKER]

            # Count sentiments
            pos_count = len(btc_sentiments["positive"])
            neg_count = len(btc_sentiments["negative"])
            neu_count = len(btc_sentiments["neutral"])
            total_replies = pos_count + neg_count + neu_count

            # Calculate average sentiment scores and reply lengths
            pos_score = (
                np.mean([x[2] for x in btc_sentiments["positive"]])
                if pos_count > 0
                else 0
            )
            neg_score = (
                np.mean([x[2] for x in btc_sentiments["negative"]])
                if neg_count > 0
                else 0
            )
            neu_score = (
                np.mean([x[2] for x in btc_sentiments["neutral"]])
                if neu_count > 0
                else 0
            )

            # Calculate average reply lengths for each sentiment
            pos_lengths = (
                np.mean([len(x[1]) for x in btc_sentiments["positive"]])
                if pos_count > 0
                else 0
            )
            neg_lengths = (
                np.mean([len(x[1]) for x in btc_sentiments["negative"]])
                if neg_count > 0
                else 0
            )
            neu_lengths = (
                np.mean([len(x[1]) for x in btc_sentiments["neutral"]])
                if neu_count > 0
                else 0
            )

            tweet_data.append(
                {
                    "date": pd.to_datetime(tweet["created_at"]).normalize(),
                    "tweet_text": tweet["text"],
                    "author": tweet["user"]["username"],
                    "followers": tweet["user"]["followers_count"],
                    "reply_count": tweet["reply_count"],
                    "retweet_count": tweet["retweet_count"],
                    "like_count": tweet["like_count"],
                    "ticker": TICKER,
                    "main_post_label": main_post[3],
                    "main_post_score": main_post[2],
                    "positive_replies": pos_count,
                    "negative_replies": neg_count,
                    "neutral_replies": neu_count,
                    "total_replies": total_replies,
                    "avg_positive_score": pos_score,
                    "avg_negative_score": neg_score,
                    "avg_neutral_score": neu_score,
                    "avg_positive_length": pos_lengths,
                    "avg_negative_length": neg_lengths,
                    "avg_neutral_length": neu_lengths,
                }
            )

# Create DataFrame
sentiment_df = pd.DataFrame(tweet_data)
sentiment_df = sentiment_df.sort_values("date")
sentiment_df['date'] = pd.to_datetime(sentiment_df['date']).dt.tz_localize(None)


sentiment_df.head()

In [ ]:
# Convert price_data date to datetime for merging
price_data['date'] = pd.to_datetime(price_data['date'])

# Create a function to find the closest future price date
def get_next_price(tweet_date, price_dates):
    future_dates = price_dates[price_dates > tweet_date]
    if len(future_dates) > 0:
        return future_dates.min()
    return None

# Add column with next price date
sentiment_df['price_date'] = sentiment_df['date'].apply(
    lambda x: get_next_price(x, price_data['date'])
)

# Merge with price data based on the calculated price date
sentiment_df = sentiment_df.merge(
    price_data,
    left_on='price_date',
    right_on='date',
    how='left',
    suffixes=('', '_price')
)

# Drop the temporary price_date column and redundant date_price column
sentiment_df = sentiment_df.drop(['price_date', 'date_price'], axis=1)

# Sort by original date
sentiment_df = sentiment_df.sort_values('date')


In [ ]:
# sentiment_df.to_csv("../data/sentiment_data.csv", index=False)

In [ ]:
sentiment_df["numeric_label"] = sentiment_df.apply(
    lambda x: (
        +x["main_post_score"]
        if x["main_post_label"] == "positive"
        else (
            x["main_post_score"] / 3
            if x["main_post_label"] == "neutral"
            else -x["main_post_score"]
        )
    ),
    axis=1,
)

In [ ]:
sentiment_df.head()

,date,tweet_text,author,followers,reply_count,retweet_count,like_count,ticker,main_post_label,main_post_score,...,total_replies,avg_positive_score,avg_negative_score,avg_neutral_score,avg_positive_length,avg_negative_length,avg_neutral_length,price,change,numeric_label
0,2024-01-01,#BTC: You are here \n\nHappy New Year! https:/...,AlemzadehC,55094,88,222,1245,btc,positive,0.952493,...,20,0.937525,0.643284,0.703171,62.300000,115.5,90.500000,44183.4,4.52,0.952493
16,2024-01-01,"#Bitcoin 🚨 Send it straight to $48,000 😈\n\n#B...",KevinSvenson_,160207,82,194,1128,btc,neutral,0.750014,...,4,0.780954,0.000000,0.827695,24.333333,0.0,32.000000,44183.4,4.52,0.250005
15,2024-01-01,BTC up 4%. \n#SOLALTS up 7% to 27% \n#Solana h...,invest_answers,259497,50,35,468,btc,neutral,0.696106,...,6,0.830440,0.000000,0.740171,51.333333,0.0,50.666667,44183.4,4.52,0.232035
14,2024-01-01,GM 🌞\nIt is 2024 and crypto is up again !!!\n$...,Mrcapitalv,6225,93,4,99,btc,positive,0.718089,...,20,0.931919,0.000000,0.767155,51.333333,0.0,19.500000,44183.4,4.52,0.718089
13,2024-01-01,#bitcoin is ripping on news that #bitcoin expe...,BitPaine,39464,68,58,847,btc,neutral,0.586407,...,20,0.785715,0.892843,0.654047,43.750000,49.0,74.769231,44183.4,4.52,0.195469


In [ ]:
reputation_df = pd.DataFrame(columns=["author", "reputation", "engagement", "accuracy"])
reputation_df.head()

# Calculate reputation scores for each author
# process one day at a time, in chronological order
for date in sentiment_df["date"].unique():
    print(date)
    daily_data = sentiment_df[sentiment_df["date"] == date]

    # Calculate reputation scores for each author
    for author in daily_data["author"].unique():
        author_data = daily_data[daily_data["author"] == author]
        author_data.head()

        engagement = (
            4 * author_data["reply_count"].sum()
            + 2 * author_data["retweet_count"].sum()
            + author_data["like_count"].sum()
        ) / 7
        accuracy = (
            1
            if (
                author_data["numeric_label"].mean() > 0
                and author_data["change"].mean() > 0
            )
            else 0
        )

        total_replies = author_data["total_replies"].sum()
        if total_replies == 0:
            debate_quality = 0  # No replies means no debate
        else:
            # Create array of sentiment values (-1 for negative, 0 for neutral, 1 for positive)
            sentiments = (
                -1 * int(author_data["negative_replies"].sum()),
                int(author_data["neutral_replies"].sum())
                + 0.4,  # Neutral sentiment is less impactful, center around 0
                1 * int(author_data["positive_replies"].sum()),
            )

            # Higher std dev means more spread in opinions (better debate)
            # Normalize to 0-1 range (max std dev for [-1,0,1] is ~0.816)
            debate_quality = np.std(sentiments) / 0.816 if sentiments else 0

        # Normalize each component using sigmoid function for smoother scaling
        norm_engagement = 2 / (1 + np.exp(-engagement/5)) - 1  # Centers around 0, range [-1,1]
        norm_debate = 2 / (1 + np.exp(-debate_quality/3)) - 1   # Centers around 0, range [-1,1]
        
        # Combine normalized components with weights

        reputation = 0.4 * norm_engagement + 4 * accuracy + 5 * norm_debate
        # Scale to positive range
        reputation = (reputation + 1) * 2.5  # Maps [-1,1] to [0,5]

        print(author, reputation, norm_engagement, accuracy, debate_quality)

        # Update or add new row for author with weight decay
        if author in reputation_df["author"].values:
            # Update existing entry with weight decay
            idx = reputation_df.index[reputation_df["author"] == author][0]
            decay_factor = 0.8  # Previous data has 30% weight decay

            # Blend new and historical values
            reputation_df.at[idx, "reputation"] = (
                decay_factor * reputation_df.at[idx, "reputation"]
                + (1 - decay_factor) * reputation
            )
            reputation_df.at[idx, "engagement"] = (
                decay_factor * reputation_df.at[idx, "engagement"]
                + (1 - decay_factor) * norm_engagement
            )
            reputation_df.at[idx, "accuracy"] = (
                decay_factor * reputation_df.at[idx, "accuracy"]
                + (1 - decay_factor) * accuracy
            )
            reputation_df.at[idx, "debate_quality"] = (
                decay_factor * reputation_df.at[idx, "debate_quality"]
                + (1 - decay_factor) * norm_debate
            )
        else:
            # Add new entry
            new_row = pd.DataFrame(
                {
                    "author": [author],
                    "reputation": [reputation],
                    "engagement": [engagement],
                    "accuracy": [accuracy],
                    "debate_quality": [debate_quality],
                }
            )
            reputation_df = pd.concat([reputation_df, new_row], ignore_index=True)
    # Apply 1% reduction to all reputation scores
    reputation_df["reputation"] = reputation_df["reputation"] * 0.99

2024-01-01 00:00:00
AlemzadehC 23.445413190780208 1.0 1 6.519587367078433
KevinSvenson_ 16.565447089783085 1.0 1 1.5020242084027973
invest_answers 24.893492575940137 1.0 1 9.217192664163697
Mrcapitalv 23.15849931523404 0.9999977228098007 1 6.1616691617532044
BitPaine 24.490391886393105 0.9999999999999991 1 8.234224175749421
AxisAlive 22.942721526685666 0.9998392249924117 1 5.912910824419029
tonythebullBTC 18.083963971426012 0.9996985792838806 1 2.3079161587950225
OrdiswapLabs 22.110615407447114 0.9999999999999738 1 5.074575213486777
Stamp_Seed 14.546292951177664 0.9995239311279736 1 0.5036285950138053
TheBTCTherapist 21.387454382442513 1.0 1 4.458419493347142
Nyteowl_YT 23.158362218631837 0.9998606262076015 1 6.1616691617532044
CoinstoreExc 25.984404404529737 1.0 1 22.137055657898717
charliebilello 21.007169431707162 1.0 1 4.164263051429627
6thmanTO 23.285479361245546 0.9993292997390673 1 6.316791431397944
2024-01-02 00:00:00
BritishHodl 13.648629898843677 0.9999999999999849 0 6.795303

/tmp/ipykernel_26541/4141694723.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  reputation_df = pd.concat([reputation_df, new_row], ignore_index=True)


fencer_gg 6.338169248979627 1.0 0 1.3864838846795051
olvelez007 13.591900688810483 0.9999988196629828 0 6.716262717177978
merts_eth 11.101732034987153 0.9999930605279557 0 4.2357782342535035
btcNLNico 11.756541219414164 1.0 0 4.762448248950873
davincithreads 13.584201794778874 0.9995239311279736 0 6.70631711314743
njelsalvador 5.68348110445697 0.9999710437486318 0 1.0589445585633759
ronin21btc 15.51715228760192 1.0 0 11.782280808776306
Tele_Swap 14.814740405366834 1.0 0 9.001028747788695
MEonBTC 5.172475522556096 0.9976442321188728 0 0.8087822660630446
TomCrownCrypto 11.688926452807372 0.9990277625419501 0 4.70605255683486
StevenNerayoff 13.676301699778485 0.9999999999974074 0 6.834481169400124
scottmelker 12.764027520881447 1.0 0 5.71778699572637
TylerSCrypto 15.16037303295889 0.999999999615194 0 10.078534158491825
2024-01-06 00:00:00
MicahPW20 13.795412914422322 0.9999910256766651 0 7.0080706908520725
fencer_gg 5.174831290437223 1.0 0 0.8087822660630446
BTC_DeGods 5.174057633935938 0

WhaleFUD 13.225563248057327 0.9999999998412985 0 6.2423861094421795
BTCsessions 12.271037744934937 0.9999998402586738 0 5.22364946508032
murtaza 15.207849865702928 0.9999999999999996 0 10.259043890419779
theKoolCaleb 14.034500959365541 1.0 0 7.383740867774512
MinisterOfNFTs 15.938439513612835 0.9999999999993032 0 18.012436313098917
dot_swap 13.027003968337388 1.0 0 6.008096833611189
2024-01-10 00:00:00
CalebFranzen 24.22555205511091 1.0 1 7.715291090083685
LucaNetz 22.75303840369036 0.9999999999824154 1 5.706101336677852
mannythehitman 20.369211651135835 0.9999999999997646 1 3.7063059549718154
KevinSvenson_ 17.812370065886928 0.9999999999999996 1 2.158471374774985
_RichardTeng 25.679657380473397 1.0 1 13.033030459221745
TO 25.837273944145313 1.0 1 15.084097912214736
HeyItsMeTheDev 23.181108565909305 0.9999999999999938 1 6.188691785753473
0xChar 22.76402730831172 0.9999997874302746 1 5.71778699572637
TheBTCTherapist 15.979755329684027 0.9999999999999991 1 1.2062763938099184
DeFi 23.4098

In [ ]:
reputation_df.describe()

,reputation,engagement,debate_quality
count,221.000000,221.000000,221.000000
mean,14.236942,230.297233,5.340354
std,5.649788,508.790158,3.196083
min,2.891590,14.856182,0.000000
25%,10.849760,53.000000,2.993995
50%,13.512144,101.428571,5.752702
75%,19.263253,195.857143,7.542306
max,24.573597,5814.142857,22.137056


In [ ]:
reputation_df.sort_values("reputation", ascending=False).head()


,author,reputation,engagement,accuracy,debate_quality
207,Stelow_1,24.573597,32.428571,1,9.779381
206,almightyrainz,24.558662,23.428571,1,9.779381
199,DesLucrece,24.123799,51.285714,1,9.129879
202,shakepay,23.794167,504.000000,1,8.302841
204,BVMnetwork,23.472618,5814.142857,1.0,7.516125


In [ ]:
reputation_df.to_csv("../data/reputation_data.csv", index=False)
